# Получение предсказаний обученной модели



## Библиотеки

In [133]:
import pandas as pd
import numpy as np
import json
import yaml

from sklearn.metrics import f1_score

from bert_inference import run_inference
from bert_model import BertForSequenceClassification


In [186]:
#загрузка конфига
with open("config.yml", "r") as yamlfile:
    cfg = yaml.safe_load(yamlfile)
#загрузка предобученной модели
bert_model = BertForSequenceClassification(
        pretrained_model_name='DeepPavlov/rubert-base-cased',
        num_labels=cfg['model']['num_classes'],
        dropout=cfg['model']['dropout'],
        inference=True
        )


Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Предсказание для 1 документа

In [188]:
#загрузка для маппинга классов
with open('labels.json', 'r', encoding='utf8') as f:
    kind_names = json.load(f)
kind_names = {int(i) : v for i, v in kind_names.items()}
kind_names_rev = {v : int(i) for i, v in kind_names.items()}
kind_names_rev



{'arrangement': 0,
 'application': 1,
 'proxy': 2,
 'contract': 3,
 'act': 4,
 'order': 5,
 'determination': 6,
 'statute': 7,
 'contract offer': 8,
 'invoice': 9,
 'bill': 10}

In [189]:

kind_names


{0: 'arrangement',
 1: 'application',
 2: 'proxy',
 3: 'contract',
 4: 'act',
 5: 'order',
 6: 'determination',
 7: 'statute',
 8: 'contract offer',
 9: 'invoice',
 10: 'bill'}

In [137]:
#пример для сабмита на лб
df_submit = pd.read_csv('submission.csv', sep=';')
df_submit


,document_id,class_id
0,-6246857835391756489,6
1,-3556905392020671700,6
2,7555384868749394362,6
3,-8330052827067299541,6
4,-6432527667337110209,6
...,...,...
258,-6114656070195605344,4
259,-5651493222750030564,4
260,-2419354930962108748,4
261,8995366856044166079,4


In [190]:
#загрузка тествых данных
df_test = pd.read_csv('dataset.csv', sep='|')
df_test


,document_id,document_text
0,-1704473130726551677,Приложение №1 к договору подряда № 2 от 25.07....
1,-8755312099564304902,Директору ООО «Вершки-Корешки» Головину Олегу ...
2,4407537979341024747,Приложение к договору ренты № 15 от 25.07.2024...
3,-2949797178567945855,Приложение ...
4,-3027899216558317249,ООО «Электросталь»Приказ № 66/ВО выводе из экс...
...,...,...
258,3405154703345778066,ДоверенностьГ. Курган ...
259,4748799364308515784,ООО «Невада холдинг»Приказ № 123О назначении о...
260,8934602722747847927,Приложение № 2к Договору №____________ от «___...
261,-6862048199031279271,Договор с покупателемг. Москва ...


In [191]:
#проверка предсказания для 1 документа
doc = df_test['document_text'][3]
most_confident, _, _, _ = run_inference(doc, 'cpu', bert_model)


Inference:   0%|          | 0/1 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/don/DataScience/.venv_ds/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Inference: 100%|██████████| 1/1

In [194]:
print('предсказанный класс документа -', kind_names[most_confident])


предсказанный класс документа - contract offer


## Предсказание для списка документов


In [195]:
#списки для предсказаний моделью
most_confident_label = list()
most_confident_labels = list()
getting_confidences_args = list()
probabilities = list()


In [196]:
#словарь для обновленного маппинга для отправки на ЛБ
class_dict = {
    1 : 'proxy',
    2 : 'contract',
    3 : 'act',
    4 : 'application',
    5 : 'order',
    6 : 'invoice',
    7 : 'bill',
    8 : 'arrangement',
    9 : 'contract offer',
    10 : 'statute',
    11 : 'determination'
}

class_dict = {v : int(i) for i, v in class_dict.items()}
class_dict


In [198]:
%%time
#Предсказание для всего датасета
for i in df_test['document_text'].values:
    most_confident, most_labels, getting_args_, probab= run_inference(i, 'cpu', bert_model)

    most_confident_label.append(most_confident)
    most_confident_labels.append(most_labels)
    getting_confidences_args.append(getting_args_)
    probabilities.append(probab)


Inference:   0%|          | 0/1 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/don/DataScience/.venv_ds/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Inference: 100%|██████████| 1/1

CPU times: user 3min 22s, sys: 1.41 s, total: 3min 24s
Wall time: 2min 5s


In [205]:
  #соеденение для предсказаний
  df_test['predict'] = most_confident_label
  df_test


,document_id,document_text,predict,predict_text
0,-1704473130726551677,Приложение №1 к договору подряда № 2 от 25.07....,4,act
1,-8755312099564304902,Директору ООО «Вершки-Корешки» Головину Олегу ...,1,application
2,4407537979341024747,Приложение к договору ренты № 15 от 25.07.2024...,4,act
3,-2949797178567945855,Приложение ...,8,contract offer
4,-3027899216558317249,ООО «Электросталь»Приказ № 66/ВО выводе из экс...,5,order
...,...,...,...,...
258,3405154703345778066,ДоверенностьГ. Курган ...,2,proxy
259,4748799364308515784,ООО «Невада холдинг»Приказ № 123О назначении о...,5,order
260,8934602722747847927,Приложение № 2к Договору №____________ от «___...,0,arrangement
261,-6862048199031279271,Договор с покупателемг. Москва ...,0,arrangement


In [206]:
#старый маппинг для перевода лейблов в текстовый вариант
df_test['predict_text'] = df_test['predict'].map(kind_names)
df_test


,document_id,document_text,predict,predict_text
0,-1704473130726551677,Приложение №1 к договору подряда № 2 от 25.07....,4,act
1,-8755312099564304902,Директору ООО «Вершки-Корешки» Головину Олегу ...,1,application
2,4407537979341024747,Приложение к договору ренты № 15 от 25.07.2024...,4,act
3,-2949797178567945855,Приложение ...,8,contract offer
4,-3027899216558317249,ООО «Электросталь»Приказ № 66/ВО выводе из экс...,5,order
...,...,...,...,...
258,3405154703345778066,ДоверенностьГ. Курган ...,2,proxy
259,4748799364308515784,ООО «Невада холдинг»Приказ № 123О назначении о...,5,order
260,8934602722747847927,Приложение № 2к Договору №____________ от «___...,0,arrangement
261,-6862048199031279271,Договор с покупателемг. Москва ...,0,arrangement


In [207]:
#новый маппинг лейблов для предсказания на ЛБ
df_test['class_id'] = df_test['predict_text'].map(class_dict)
df_test


,document_id,document_text,predict,predict_text,class_id
0,-1704473130726551677,Приложение №1 к договору подряда № 2 от 25.07....,4,act,3
1,-8755312099564304902,Директору ООО «Вершки-Корешки» Головину Олегу ...,1,application,4
2,4407537979341024747,Приложение к договору ренты № 15 от 25.07.2024...,4,act,3
3,-2949797178567945855,Приложение ...,8,contract offer,9
4,-3027899216558317249,ООО «Электросталь»Приказ № 66/ВО выводе из экс...,5,order,5
...,...,...,...,...,...
258,3405154703345778066,ДоверенностьГ. Курган ...,2,proxy,1
259,4748799364308515784,ООО «Невада холдинг»Приказ № 123О назначении о...,5,order,5
260,8934602722747847927,Приложение № 2к Договору №____________ от «___...,0,arrangement,8
261,-6862048199031279271,Договор с покупателемг. Москва ...,0,arrangement,8


In [208]:
#итоговый формат сабмита
submit = df_test[['document_id', 'class_id']]
submit


,document_id,class_id
0,-1704473130726551677,3
1,-8755312099564304902,4
2,4407537979341024747,3
3,-2949797178567945855,9
4,-3027899216558317249,5
...,...,...
258,3405154703345778066,1
259,4748799364308515784,5
260,8934602722747847927,8
261,-6862048199031279271,8


In [169]:

#сохранение сабмита
submit.to_csv('submit_5.csv', sep=';', index=False)
